# Bayesian Mixture of Gaussians in edward2

A similar tutorial is written with tfp:
https://github.com/tensorflow/probability/blob/master/tensorflow_probability/examples/jupyter_notebooks/Bayesian_Gaussian_Mixture_Model.ipynb

In [9]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="7"

In [10]:
from functools import partial
import matplotlib.pyplot as plt
import tensorflow as tf
Variable = tf.Variable
import tensorflow_probability as tfp
tfd = tfp.distributions
ed = tfp.edward2

print('TensorFlow version {}, TF Probability version {}.'.format(tf.__version__, tfp.__version__))

TensorFlow version 1.10.0, TF Probability version 0.4.0.


In [26]:
def mog_model(alpha, d=2, k=3):
    pi = ed.Dirichlet(concentration=[alpha, alpha])
    mu = ed.Normal(loc=tf.zeros([d, k]), scale=tf.ones([d, k]))
    return pi, mu
model = mog_model(0.5)
with tf.Session() as sess:
    print(sess.run(model))

(array([0.9686755, 0.0313245], dtype=float32), array([[-0.71308744,  0.17273177,  0.71759415],
       [-1.7080716 ,  0.3241706 ,  1.2325232 ]], dtype=float32))
